# Demoing GPT to BlogPost

In [77]:
!pip install --upgrade openai

In [78]:
from openai import OpenAI
import random
import os
from dotenv import load_dotenv
import json

load_dotenv()
gpt = os.getenv('gpt_token')
org = os.getenv('gpt_org')
client = OpenAI(api_key=gpt, organization=org)

def read_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return [json.loads(line) for line in file]

examples = read_jsonl("../example_docs/blog_posts.jsonl")

seo_terms = []
with open('../example_docs/seo_key_topics.txt', 'r') as file:
    seo_terms = [line.strip() for line in file]

seo_topics = random.choice(seo_terms)

sample = []
for i in range(0,9):
    index = random.randint(0, len(examples))
    sample.append(examples[index])
    
examples_str = '\n'.join(json.dumps(example, indent=4) for example in sample[0:8])

master_prompt = f"""
            You are a real-estate blog writer. Given examples of blog posts and also extra data accumulated from external
            sources, you will write a unique real estate blog post, making use of SEO keywords to make the blog post stand out. In the blog post itself,
            you will provide:
                - URL: a url for the blog post that follows SEO standards
                - Title: A title of what will covered in the blog post
                - Content: The content of the blog post. Write the blog post in markdown. You do not have to tell me it is in markdown, just please
                           write the blog post in markdown. It should consist of at least 5000 words. Make sure that there is a lot to say 
                           within categories and subcategories, making it an information read for the reader
                - SEO Terms: A list of the SEO keywords used in the blog post
            
            Depending on what is asked of you, based on how many blogposts the client requests from, you will always default to one blog post.
            If the user asks for a specific amount, then provide that amount of blogposts in a listed array. However, in the output, you should respond
            with a json object of this structure:
                "url": "The blog post url",
                "title": "The blog post title",
                "content": "The blog post content",
                "seo_terms": ["keyword1", "keyword2"]
            Please follow the strucure and put a comma after every single key field. The content should be one whole string and any 
            kind of special character like \n for newline should be integrated within the content string to provide for that newline.
            You do not have to tell me or put into a comment that this is a json, the user will already know its a json.
            Please make sure that you follow the structure above and that something like seo terms is not found within "content", but should rather be 
            in the "seo_terms" key of the object body. I REPEAT DO NOT WRITE THE SEO TERMS WITHIN THE CONTENT IN MARKDOWN. IT SHOULD ONLY BE SEEN
            WITHIN "seo_terms". Please make sure that you are not using external links and link them inside the blog post you generate. 
            Please do not mention anything promotional or any footers regarding where the blog post is getting its data from, 
            how it's written, or anything not related to what a blog post is supposed to write. Please do not write a table of contents. 
            You are simply writing content. Please also make use of real estate SEO terms from example blog posts that may be given from the user.
            """

user_prompt = f"""
            Hello, I would like you to help me generate a unique real estate blog post. May you please write a blog with this topic: 
            {seo_topics}
            Here are examples of how real estate blog posts that I saw and would hope you would be able to come up with something unique that consists 
            of similar topics of the examples: {examples_str}. Follow the structure of the examples, and use some context from the examples, but use 
            the context and create your own blog post. And please provide a lot of numbers and statistics. Write the blog around the current month of 
            August 2024. Make the blog location specific. The blog should based in New York, NY. Make sure that SEO keywords are also seen frequently.
            """

tools = [
    {
        "type": "function",
        "function": {
            "name": "generate_blogpost",
            "description": "Generate a detailed real estate blog post based on user request and make use of SEO terms as keywords when constructing url, title and content",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "Includes url for real estate blogpost that engages SEO keywords of real estate based on the topic.",
                    },
                    "title": {
                        "type": "string",
                        "description": "Includes title for real estate blogpost that engages SEO keywords of real estate based on the topic.",
                    },
                    "content": {
                        "type": "string",
                        "description": "Includes content for real estate blogpost that engages SEO keywords of real estate based on the topic. Can you use markdown for this",
                    },
                    "seo_terms": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Provide the list of SEO terms used when creating the real estate blogpost",
                    }
                },
                "required": [
                    "url",
                    "title",
                    "content",
                    "seo_terms",
                ]
            }
        }
    }
]
# print(user_prompt)
response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": master_prompt},
                    {"role": "user", "content": user_prompt},
                ],
                tools=tools,
                tool_choice="required",
                max_tokens=4096,
                temperature=0.7,
            )

# SEO Agent next - follow SEO structure
# Humanizer Agent
# Reflection of Agent
# Pushing into Wordpress

In [79]:
response_dump = response.model_dump()
choices = response_dump.get("choices",[])
if choices:
    message = choices[0].get("message", {})
    tool_calls = message.get("tool_calls", [])
    if tool_calls:
        function = tool_calls[0].get("function", {})
        arguments = function.get("arguments", {})
        if arguments:
            response_json = arguments
try:
    data = json.loads(response_json)
except:
    print("fail")

In [80]:
print("Topic of choice: ", seo_topics)
print(data["content"])

Topic of choice:  Real Estate Mania: Expert Insights and Market Predictions
## Real Estate Mania: Expert Insights and Market Predictions in New York, NY

August 2024 has brought a whirlwind of activity to the New York City real estate market. As the city that never sleeps continues to evolve, real estate professionals and investors are keenly observing trends, analyzing data, and making predictions about the future. In this comprehensive blog post, we'll delve into the current state of the NYC real estate market, explore expert insights, and discuss market predictions that could shape the landscape for years to come.

### The Current State of the NYC Real Estate Market

The New York City real estate market has always been a dynamic and ever-changing entity. As of August 2024, several key trends and statistics have emerged:

1. **Rising Property Values**: Property values in NYC have seen a significant increase over the past year. According to recent data, the median home price in Manhat

In [81]:
master_prompt_2 = """
        You are a real estate blog post reviewer. Your job is to read through the contents of real estate blog posts and provide suggestions
        on how to make it better in terms of content, SEO, and providing more stats or information that will make the real estate blogpost more 
        accurate to current data.
"""

user_prompt_2 = f"""
        I wrote a real estate blog post based on this topic: {seo_topics}. Here is the blog post that I wrote: {data["content"]}.
        Can you give me suggestions to make it better in terms of content, writing style, and better SEO suggestions.
"""

tool2 = [
    {
        "type": "function",
        "function": {
            "name": "generate_edits",
            "description": "Generate a detailed list of suggestions to make the real estate blog post provided by the user better when it comes to content, data, and SEO rules",
            "parameters": {
                "type": "object",
                "properties": {
                    "suggestions": {
                        "type": "string",
                        "description": "List of suggestions for improvements on the real estate blog post"
                    }
                },
                "required": [
                    "suggestions",
                ]
            }
        }
    }
]

response2 = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": master_prompt_2},
                    {"role": "user", "content": user_prompt_2},
                ],
                tools=tool2,
                tool_choice="required",
                max_tokens=4096,
                temperature=0.7,
            )

In [82]:
response_dump2 = response2.model_dump()
choices2 = response_dump2.get("choices",[])
if choices2:
    message2 = choices2[0].get("message", {})
    tool_calls2 = message2.get("tool_calls", [])
    if tool_calls2:
        function2 = tool_calls2[0].get("function", {})
        arguments2 = function2.get("arguments", {})
        if arguments2:
            response_json2 = arguments2
try:
    data2 = json.loads(response_json2)
    print(data2['suggestions'])
except:
    print("fail")

1. **Content Enrichment**: 
   - Add more granular data like neighborhood-specific trends in Manhattan, Brooklyn, and Queens. 
   - Incorporate case studies or real-life examples of recent big transactions or property developments. 
   - Include expert opinions on emerging neighborhoods or hidden gems in NYC.

2. **SEO Optimization**: 
   - Use keyword-rich headings and subheadings (e.g., "NYC Real Estate Market Trends August 2024"). 
   - Include internal links to other relevant blog posts on your site and external links to authoritative sources for credibility.
   - Add meta descriptions and alt text for images to improve search engine visibility.

3. **Data and Statistics**: 
   - Provide sources for all data points mentioned (e.g., "According to Zillow..."). 
   - Include comparative data from previous years to show trends over time.
   - Add visual aids like graphs or charts to make the data more digestible.

4. **Writing Style**: 
   - Use a more engaging and conversational tone 

In [87]:
master_prompt_3 = """
    You are a skilled real estate blog post writer tasked with crafting the final draft of a blog post. You will receive the first draft of the 
    blog post along with a set of suggestions provided by a real estate blog post reviewer. Your task is to carefully review the first draft 
    and fully incorporate the suggestions provided to create a polished final draft. Enhance the clarity, engagement, and natural flow of 
    the content, ensuring it reads smoothly and appeals to the target audience. Remember to humanize the content, making it feel conversational and 
    approachable while retaining the key messages and SEO elements. There should be a noticeable difference from the original draft.
"""


user_prompt_3 = f"""
I wrote an initial draft of a real estate blog post on the topic: {seo_topics}. I want to improve this draft based on suggestions I received from a real estate blog post reviewer.

Here is the first draft of the blog post:
{data['content']}

Here are the suggestions:
{data2['suggestions']}

Please incorporate these suggestions into the final draft, enhancing readability, tone, and structure. Ensure the final draft is more engaging, flows naturally, and is well-aligned with the topic and SEO objectives.
"""


tools_3 = [
    {
        "type": "function",
        "function": {
            "name": "generate_reviewed_blogpost",
            "description": "Generate a final draft of a real estate blogpost that was provided by the user and rewrite it with the suggestions they provided",
            "parameters": {
                "type": "object",
                "properties": {
                    "url": {
                        "type": "string",
                        "description": "Includes url for real estate blogpost that engages SEO keywords of real estate based on the topic. Only provide everything after the domain name",
                    },
                    "title": {
                        "type": "string",
                        "description": "Includes title for real estate blogpost that engages SEO keywords of real estate based on the topic.",
                    },
                    "content": {
                        "type": "string",
                        "description": "Includes content for real estate blogpost that engages SEO keywords of real estate based on the topic. Can you use markdown for this. Only provide the content of blog post do not add your own thoughts outside of the blog posts",
                    },
                    "seo_terms": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "Provide the list of SEO terms used when creating the real estate blogpost",
                    }
                },
                "required": [
                    "url",
                    "title",
                    "content",
                    "seo_terms",
                ]
            }
        }
    }
]

response3 = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": master_prompt_3},
                    {"role": "user", "content": user_prompt_3},
                ],
                tools=tools_3,
                tool_choice="required",
                max_tokens=4096,
                temperature=1.0,
            )

In [88]:
response_dump3 = response3.model_dump()
choices3 = response_dump3.get("choices",[])
if choices3:
    message3 = choices3[0].get("message", {})
    tool_calls3 = message3.get("tool_calls", [])
    if tool_calls3:
        function3 = tool_calls3[0].get("function", {})
        arguments3 = function3.get("arguments", {})
        if arguments3:
            response_json3 = arguments3
try:
    print(response_json3)
    data3 = json.loads(response_json3)
except:
    print("fail")

{"url": "real-estate-mania-expert-insights-and-market-predictions", "title": "Real Estate Mania: Expert Insights and Market Predictions in New York, NY", "content": "## NYC Real Estate Market Trends August 2024\n\nAugust 2024 has brought a whirlwind of activity to the New York City real estate market. The city that never sleeps continues to evolve, with keen insights from real estate professionals and investors shaping its future. In this comprehensive blog post, we'll delve into the current state of the NYC real estate market, explore expert insights, and discuss market predictions that could shape the landscape for years to come.\n\n### The Current State of the NYC Real Estate Market\n\nThe New York City real estate market has always been a dynamic and ever-changing entity. As of August 2024, several key trends and statistics have emerged:\n\n1. **Rising Property Values**: Property values in NYC have seen a significant increase over the past year. According to recent data from Zillow

In [89]:
print(data3['content'])

## NYC Real Estate Market Trends August 2024

August 2024 has brought a whirlwind of activity to the New York City real estate market. The city that never sleeps continues to evolve, with keen insights from real estate professionals and investors shaping its future. In this comprehensive blog post, we'll delve into the current state of the NYC real estate market, explore expert insights, and discuss market predictions that could shape the landscape for years to come.

### The Current State of the NYC Real Estate Market

The New York City real estate market has always been a dynamic and ever-changing entity. As of August 2024, several key trends and statistics have emerged:

1. **Rising Property Values**: Property values in NYC have seen a significant increase over the past year. According to recent data from Zillow, the median home price in Manhattan has reached $1.2 million, reflecting a 10% year-over-year growth. Brooklyn and Queens have also experienced notable price increases, with

In [86]:
print(data["content"])

## Real Estate Mania: Expert Insights and Market Predictions in New York, NY

August 2024 has brought a whirlwind of activity to the New York City real estate market. As the city that never sleeps continues to evolve, real estate professionals and investors are keenly observing trends, analyzing data, and making predictions about the future. In this comprehensive blog post, we'll delve into the current state of the NYC real estate market, explore expert insights, and discuss market predictions that could shape the landscape for years to come.

### The Current State of the NYC Real Estate Market

The New York City real estate market has always been a dynamic and ever-changing entity. As of August 2024, several key trends and statistics have emerged:

1. **Rising Property Values**: Property values in NYC have seen a significant increase over the past year. According to recent data, the median home price in Manhattan has reached $1.2 million, reflecting a 10% year-over-year growth. Brookl